<a href="https://colab.research.google.com/github/DmitriyDikunov/python-4-7/blob/main/%D0%9B%D0%90%D0%917%D0%94%D1%96%D0%BA%D1%83%D0%BD%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install scikit-surprise
from surprise import Dataset
data = Dataset.load_builtin('ml-100k')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357263 sha256=b7d5f8a81d5bc868784719a08bfff867bf4627e746d355634b546c7a3128a210
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise
Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [4]:
import pandas as pd
from surprise import Dataset
raw_data = Dataset.load_builtin('ml-100k')
trainset = raw_data.build_full_trainset()
df = pd.DataFrame(trainset.all_ratings(), columns=['user_id', 'item_id', 'rating'])
df.head(5)


,user_id,item_id,rating
0,0,0,3.0
1,0,528,4.0
2,0,377,4.0
3,0,522,3.0
4,0,431,5.0


In [5]:
from surprise import SVD, KNNBasic
algo_svd = SVD()
algo_knn = KNNBasic()



In [6]:
from surprise import accuracy
from surprise.model_selection import cross_validate
cv_svd = cross_validate(algo_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
cv_knn = cross_validate(algo_knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9310  0.9298  0.9335  0.9515  0.9343  0.9360  0.0079  
MAE (testset)     0.7340  0.7330  0.7345  0.7501  0.7368  0.7377  0.0063  
Fit time          1.69    4.69    1.45    1.81    1.44    2.22    1.25    
Test time         0.35    0.37    0.13    0.13    0.12    0.22    0.11    
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9811  0.9714  0.9882  0.9731  0.9761  0.9780  0.0061

In [7]:
mean_mae_svd = sum(cv_svd['test_mae']) / len(cv_svd['test_mae'])
mean_mae_knn = sum(cv_knn['test_mae']) / len(cv_knn['test_mae'])

if mean_mae_svd < mean_mae_knn:
    best_algo = algo_svd
    print("SVD is the best algorithm with MAE:", mean_mae_svd)
else:
    best_algo = algo_knn
    print("KNNBasic is the best algorithm with MAE:", mean_mae_knn)

SVD is the best algorithm with MAE: 0.7376815758749926


In [9]:
trainset = data.build_full_trainset()
best_algo.fit(trainset)
user_id = 196
items = trainset.all_items()
user_unrated_items = [item for item in items if not trainset.ur[user_id]]
predictions = [best_algo.predict(user_id, item) for item in user_unrated_items]
top_10_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]
for recommendation in top_10_recommendations:
    print(f"Item ID: {recommendation.iid}, Estimated Rating: {recommendation.est}")

In [10]:
#Завдання 2
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

ratings_dict = {
    "itemID": [1, 1, 1, 2, 2, 2],
    "userID": [9, 32, 2, 45, 32, 9],
    "rating": [3, 2, 4, 3, 1, 5],
}
import pandas as pd
df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[["userID", "itemID", "rating"]], reader)
trainset, testset = train_test_split(data, test_size=0.2)
algo = KNNBasic()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.mae(predictions)


Computing the msd similarity matrix...
Done computing similarity matrix.
MAE:  1.5000


1.5

In [11]:
#рекомендації користувача
user_id = 9
items = df['itemID'].unique()
rated_items = df[df['userID'] == user_id]['itemID']
unrated_items = [item for item in items if item not in rated_items]
predictions = [algo.predict(user_id, item) for item in unrated_items]
top_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

# Виведемо рекомендації
for recommendation in top_recommendations:
    print(f"Item ID: {recommendation.iid}, Estimated Rating: {recommendation.est}")


Item ID: 1, Estimated Rating: 2.5
Item ID: 2, Estimated Rating: 2.5


Висновок
у ході цієї лабораторної роботи було реалізовано власну рекомендаційну систему на основі алгоритму KNN з використанням бібліотеки Surprise. Спочатку було створено невеликий датасет із рецензій користувачів, на основі якого система навчалася та оцінювалася за допомогою метрики середньої абсолютної помилки (MAE). Далі було отримано рекомендації для конкретного користувача, що дало змогу спрогнозувати його потенційні вподобання щодо фільмів, які він ще не оцінював.